<a href="https://colab.research.google.com/github/tejatanush/Kaggle-NLP-with-disaster-tweets/blob/main/Kaggle_NLP_with_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Model Description:**

This model is for **Spam or Ham** to predict which tweets are Real and which are not.This is kaggle competition...
https://www.kaggle.com/competitions/nlp-getting-started
## Kaggle Competition: Natural Language Processing with Disaster Tweets
- **Position**: 238
- **Test Data Accuracy**: 82.562%
- **Model Used**: Distilbert base uncased (fine tuning)
- **Description**: This is the beginner competition in kaggle which is the biggest platform for machine learning datasets,models...etc. This platform also hosts competitions all over the world which helps to boost our knowledge by building models and compare to others.This model predicts which tweet is spam and which is not  by using NLP techniques.


# 1. Import Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# 2.Import Dataset
This dataset is provided by kaggle itself for all participants with 3 files **train.csv,test.csv,sample_submission.csv.**

Data--> https://www.kaggle.com/competitions/nlp-getting-started/data

In [4]:
train_data=pd.read_csv('train.csv')
train_data.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
test_data2=pd.read_csv('test.csv')
test_data2.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Here for test data we do not have target to check wheather the predicted is correct or not.So target is not provided. After predicting results make them to a csv file and upload in **submit prediction** then kaggle will make an accuracy test. Because of absence of target values in test.csv lets make a new data which is 20% of train.csv as validation data.

In [6]:
x=list(train_data['text'])
y=list(train_data['target'])

# 3.Split into train and validation data

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)


# 4. Install Transformers

In [8]:
!pip install transformers


# 5.Import Tokenizer and Model

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

# 6.Encode Data

In [10]:
train_encodings=tokenizer(x_train,truncation=True,padding=True)
test_encodings=tokenizer(x_test,truncation=True,padding=True)
test2_encodings=tokenizer(list(test_data2['text']),truncation=True,padding=True)

# 7.Create Dataset Objects

In [11]:
# Convert to torch tensors
import torch
train_encodings = {key: torch.tensor(val) for key, val in train_encodings.items()}
test_encodings = {key: torch.tensor(val) for key, val in test_encodings.items()}
test2_encodings={key: torch.tensor(val) for key, val in test2_encodings.items()}
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# Create a Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: tensor[idx] for key, tensor in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_datasets = Dataset(train_encodings, y_train)
test_datasets = Dataset(test_encodings, y_test)

# 8.Create Workflow

In [12]:
class TestDataset2(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: tensor[idx] for key, tensor in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [13]:
test2_dataset=TestDataset2(test2_encodings)

# 9.Set Training Arguments

In [14]:
from transformers import TFDistilBertForSequenceClassification,Trainer,TrainingArguments
training_args=TrainingArguments(output_dir='./results',
                                  num_train_epochs=2,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=16,
                                  warmup_steps=500,
                                  weight_decay=0.01,
                                  logging_dir='./logs',
                                  logging_steps=10,
                                  )


#10.Build a model

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
trainer=Trainer(model=model,
                  args=training_args,
                  train_dataset=train_datasets,
                  eval_dataset=test_datasets)


# 11. Train a model

In [16]:
trainer.train()
trainer.evaluate(test_datasets)
trainer.predict(test_datasets)
output=trainer.predict(test_datasets)[1]

Step,Training Loss
10,1.960000
20,2.797300
30,2.559400
40,2.226900
50,1.901300
60,1.238500
70,1.493400
80,0.902900
90,1.084200
100,0.766100


#12.Prediction on Test Data

In [17]:
trainer = Trainer(model=model)
predictions = trainer.predict(test2_dataset).predictions

#13.Evaluating Predictions

In [18]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,output)
cm

array([[886,   0],
       [  0, 637]])

Hurray!! Our model had predicted 100% correct on evaluation data.

#14.Save them to CSV file

In [19]:
# Convert the logits to predicted labels (0 or 1)
predicted_labels = torch.argmax(torch.tensor(predictions), axis=1).numpy()

# Prepare the final dataframe with 'id' and 'target' columns
output_df = pd.DataFrame({'id': test_data2['id'], 'target': predicted_labels})

# Save to CSV
output_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
